In [ ]:
import sys
# 添加 utils 目录到系统路径
sys.path.append('../')
import torch
from torchvision import transforms
from PIL import Image
import utils.hash_model as imgae_hash_model
device = torch.device("cuda")
# 加载图片
image_path = 'test.jpg'
image = Image.open(image_path)

# 图片预处理
# 以下是一个例子，具体的转换需要根据您模型的要求来调整
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # 假设我们使用的模型期望输入为224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image_tensor = preprocess(image).to(device).unsqueeze(0)  # 增加一个批次维度

# 加载模型
model = imgae_hash_model.HASH_Net(model_name="alexnet", bit=128, pretrained=True)
model.to(device)
model.eval()  # 设置为评估模式

# 不计算梯度，以加快推理速度
with torch.no_grad():
    hash_output = model(image_tensor)

# 将输出转换为哈希码（二进制形式）
# 这里以0为阈值进行简单的二值化为例
#hash_code = (hash_output > 0).int().numpy()
hash_code = torch.sign(hash_output)
print("哈希码:", hash_code)


In [ ]:
import torch

# 假设有一个64位整数的Tensor
hash_codes = torch.tensor([0x123456789abcdef0], dtype=torch.int64)

# 转换为位编码
def to_bit_tensor(hash_codes):
    bits = [(hash_codes >> i) & 1 for i in range(64)]
    return torch.stack(bits, dim=-1)

bit_tensor = to_bit_tensor(hash_code)
print(bit_tensor)  # 打印位编码的Tensor
